# Set environment

In [1]:
!git clone https://github.com/NirAndria/VALDOM_LLM_ENGINEERING.git
%cd VALDOM_LLM_ENGINEERING
!pip install odfpy


Cloning into 'VALDOM_LLM_ENGINEERING'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 96 (delta 20), reused 76 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (96/96), 400.33 KiB | 8.52 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/VALDOM_LLM_ENGINEERING
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=a5a964b6dc59f982eb3a1ea95e149129053f93936f4fccc966851f06fed2a64a
  Stored in directory: /root/.cache/pip/wheels/d6/1d/c8/8c29be1d73ca42d15977c75193d9f39a98499413c2838ac54c
Successfully built odfpy


# Import package

In [2]:
import os
import pandas as pd
import zipfile
from google.colab import files

In [3]:
data_path_root = "data_RAG"

# Load data

In [4]:
data = pd.ExcelFile(data_path_root + "/table/CBP7483--Tables-for-download-Feb-2022.ods", engine='odf')

In [5]:
sheet_list = data.sheet_names

In [6]:
sub_data = pd.read_excel(data, sheet_name=sheet_list[0])

In [7]:
sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
sub_data

<ipython-input-7-d38cf1def637>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)


,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
2,Election,Men,Women,Total,% women
4,1979,616,19,635,0.029921
5,1983,627,23,650,0.035385
6,1987,609,41,650,0.063077
7,1992,591,60,651,0.092166
8,1997,539,120,659,0.182094
9,2001,541,118,659,0.179059
10,2005,518,128,646,0.198142
11,2010,507,143,650,0.22
12,2015,459,191,650,0.293846


In [8]:
def is_probably_header_row(row, empty_threshold=0.2, text_threshold=0.7):
    # Check for non-empty cells
    non_empty_ratio = (row != "").mean()

    # Check how many cells are non-numeric (i.e., text-like)
    non_numeric_ratio = row.apply(lambda x: not pd.api.types.is_numeric_dtype(type(x)) and str(x).strip() != "").mean()

    # Return True if row has few empty cells and mostly text
    return (non_empty_ratio > (1 - empty_threshold)) and (non_numeric_ratio > text_threshold)


def detect_multi_level_headers(df, max_rows_to_check=3, separator=" | "):
    """
    Detect up to 3 levels of headers in the first rows of the DataFrame.
    If only one level is detected, returns it as-is. Otherwise, combines multiple levels.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the table (including header rows as data).
        max_rows_to_check (int): How many rows to check for headers (default 3).
        separator (str): String used to join multi-level headers.

    Returns:
        list: A list of header names (combined if multi-level).
        int: The number of header rows detected (useful to skip when processing data rows).
    """
    header_candidates = df.iloc[:max_rows_to_check, :]
    # Replace NaNs with empty string for safe combination
    header_candidates = header_candidates.fillna("")

    # Count how many rows are likely headers (at least 50% non-empty cells)
    header_row_mask = header_candidates.apply(lambda row: is_probably_header_row(row), axis=1)
    header_rows_count = header_row_mask.sum()
    header_rows_count = max(header_rows_count, 1)  # Ensure at least one header row

    if header_rows_count == 1:
        # Single-level header
        headers = header_candidates.iloc[0, :].tolist()
        print("Detected single-level header.")
    else:
        # Multi-level header: combine lines
        relevant_rows = header_candidates.iloc[:header_rows_count, :]
        headers = []
        for col in relevant_rows.columns:
            combined = separator.join(
                [str(val).strip() for val in relevant_rows[col] if str(val).strip() != ""]
            )
            headers.append(combined)
        print(f"Detected {header_rows_count}-level header.")

    return headers, header_rows_count



In [9]:
import pandas as pd

def is_probably_row_name_column(col, empty_threshold=0.2, text_or_date_threshold=0.7):
    # 1. Check empty cells
    non_empty_ratio = (col.astype(str).str.strip() != "").mean()

    # 2. Check for text, year-like or date-like cells
    def is_text_or_date_like(x):
        x_str = str(x).strip()
        # Check year-like
        if x_str.isdigit() and (1700 <= int(x_str) <= 2100):
            return True
        # Check date-like
        try:
            pd.to_datetime(x_str, errors="raise")
            return True
        except:
            pass
        # Otherwise: consider it text if not a pure number
        return not x_str.replace('.', '', 1).isdigit()

    text_or_date_ratio = col.apply(is_text_or_date_like).mean()

    return (non_empty_ratio > (1 - empty_threshold)) and (text_or_date_ratio > text_or_date_threshold)


def detect_multilevel_row_names(df, max_cols_to_check=2, separator=" "):
    """
    Detects up to 3 levels of row names in the first few columns of a DataFrame.
    If only one level is detected, returns that column.
    Otherwise, combines multiple levels into a single row name string per row.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the table.
        max_cols_to_check (int): How many columns from the left to check for row names.
        separator (str): Separator between combined row names.

    Returns:
        pd.Series: A series with combined row names for each row.
        int: The number of row name levels detected.
    """
    row_name_candidates = df.iloc[:, :max_cols_to_check].fillna("")

    # Check each candidate column: if >50% non-empty strings, consider row name column
    row_name_columns_mask = row_name_candidates.apply(is_probably_row_name_column, axis=0)
    row_name_levels = row_name_columns_mask.sum()
    row_name_levels = max(row_name_levels, 1)  # At least one column considered

    if row_name_levels == 1:
        # Single-level row name
        combined_names = row_name_candidates.iloc[:, 0].astype(str)
        print("Detected single-level row names.")
    else:
        # Multi-level row name: combine columns
        relevant_cols = row_name_candidates.iloc[:, :row_name_levels]
        combined_names = relevant_cols.astype(str).apply(
            lambda row: separator.join(filter(None, row.str.strip())), axis=1
        )
        print(f"Detected {row_name_levels}-level row names.")

    return combined_names.tolist(), row_name_levels


In [10]:
def extract_data_cells(df, header_levels, row_name_levels):
    """
    Extracts the data part of a table, excluding header rows and row name columns.

    Parameters:
        df (pd.DataFrame): The original DataFrame (including header and row name rows/columns).
        header_levels (int): Number of header rows detected.
        row_name_levels (int): Number of row name columns detected.

    Returns:
        pd.DataFrame: The data cells.
    """
    print(row_name_levels)
    data_cells = df.iloc[header_levels:, row_name_levels:].copy().reset_index(drop=True)

    return data_cells

In [11]:
def reconstruct_flattened_data(data_cells, combined_row_names, combined_headers, nb_header_level, nb_rowname_level, file_name="test"):
    """
    Reconstructs a tidy long-form DataFrame with columns: row_name, header, value.

    Parameters:
        data_cells (pd.DataFrame): The values extracted from the table.
        combined_row_names (pd.Series): Combined row names for each row.
        combined_headers (list): Combined headers for each column.

    Returns:
        pd.DataFrame: A long-form DataFrame with col umns: row_name, header, value.
    """
    os.makedirs("processed_table", exist_ok=True)
    file_path = os.path.join("processed_table", file_name)

    actual_row_name = combined_row_names[nb_header_level:]
    actual_header_name = combined_headers[nb_rowname_level:]

    with open(file_path + ".txt", "w", encoding="utf-8") as f:

        for idx_r in range(len(actual_row_name)):

          str_temp = "row info : " + actual_row_name[idx_r] + " | "

          for idx_c in range(len(actual_header_name)):

            str_temp += f"{actual_header_name[idx_c]}: " + str(data_cells.iloc[idx_r, idx_c])+ " | "

          str_temp += "\n"

          f.write(str_temp )

In [12]:
!pip install ezodf
import ezodf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ezodf: filename=ezodf-0.3.2-py2.py3-none-any.whl size=49002 sha256=281f120258f07c19c338732283e05295fe5bc746e15652a92108e6bbfe4eeb21
  Stored in directory: /root/.cache/pip/wheels/bb/23/3b/cce8669e20fa103fa8cd5d060b7e63ebb93cfbebd29a9e5d43
Successfully built ezodf


In [13]:
def load_ods_sheet(ods_file, sheet_name):
    ods = ezodf.opendoc(ods_file)
    sheet = ods.sheets[sheet_name] if isinstance(sheet_name, str) else ods.sheets[sheet_name]

    data = []
    for row in sheet.rows():
        values = [str(cell.value) if cell.value is not None else "" for cell in row]
        data.append(values)

    return pd.DataFrame(data)

In [14]:
for i in range(len(sheet_list)):
  #eprint(sheet_list[i])
  try:
    sub_data = pd.read_excel(data, sheet_name=sheet_list[i])
  except:

    sub_data = load_ods_sheet(data, sheet_list[i])

  sub_data = sub_data.replace(r'^\s*$', pd.NA, regex=True)
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
  if sheet_list[i] == "Length of Service":
    print(sub_data)
  header_temp, header_count_temp = detect_multi_level_headers(sub_data)
  rowname_temp, rowname_count = detect_multilevel_row_names(sub_data)
  data_cells_temp = extract_data_cells(sub_data,  header_count_temp, rowname_count)
  data_res = reconstruct_flattened_data(data_cells_temp, rowname_temp, header_temp, header_count_temp, rowname_count,file_name = sheet_list[i])


<ipython-input-14-bb06528fa64e>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
<ipython-input-14-bb06528fa64e>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
<ipython-input-14-bb06528fa64e>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
<ipython-input-14-bb06528fa64e>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_data = sub_data

Detected single-level header.
Detected single-level row names.
1
Detected 2-level header.
Detected single-level row names.
1
Detected 2-level header.
Detected single-level row names.
1
Detected 2-level header.
Detected single-level row names.
1
Detected 2-level header.
Detected single-level row names.
1
Detected single-level header.
Detected single-level row names.
1
Detected 2-level header.
Detected single-level row names.
1


<ipython-input-14-bb06528fa64e>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
<ipython-input-14-bb06528fa64e>:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
<ipython-input-14-bb06528fa64e>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
<ipython-input-14-bb06528fa64e>:10: FutureWarning: Downcasting ob

Detected 2-level header.
Detected single-level row names.
1
Detected 2-level header.
Detected single-level row names.
1
Detected single-level header.
Detected single-level row names.
1
Detected single-level header.
Detected single-level row names.
1


<ipython-input-14-bb06528fa64e>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
<ipython-input-14-bb06528fa64e>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
<ipython-input-14-bb06528fa64e>:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
<ipython-input-8-2c1e88252af8>:28: FutureWarning: Downcasting obj

              0                  1                 3                 5  \
4          <NA>       Name of MP:   Days in  service  Years in service   
6   PT30H07M29S    Peter Bottomley           16240.0  44.4626967830253   
7          <NA>   Margaret Beckett           15001.0  41.0704996577686   
8          <NA>     Barry Sheerman           14833.0  40.6105407255305   
9          <NA>     Harriet Harman           13559.0  37.1225188227242   
10         <NA>     Nicholas Brown           13335.0  36.5092402464066   
11         <NA>      Jeremy Corbyn           13335.0  36.5092402464066   
12         <NA>         Roger Gale           13335.0  36.5092402464066   
13         <NA>       Edward Leigh           13335.0  36.5092402464066   
14         <NA>       William Cash           13006.0  35.6084873374401   
15         <NA>     George Howarth           12082.0    33.07871321013   
16         <NA>       Diane Abbott           11872.0  32.5037645448323   
17         <NA>       John Redwood    

<ipython-input-14-bb06528fa64e>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_data = sub_data.dropna(how='all').dropna(axis=1, how='all').fillna(method='ffill', axis=1)


In [15]:
import shutil
from google.colab import files

# Replace 'your_subfolder' with the name of your subfolder
subfolder_name = 'processed_table'
zip_name = f'{subfolder_name}.zip'

# Compress the folder
shutil.make_archive(subfolder_name, 'zip', subfolder_name)
files.download(zip_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>